In [1]:
import sys
import os
import json
import random

# Add the Finetuning directory to path to access utils and scripts
sys.path.append('..')
from utils_ft import get_root_path, to_relative
from scripts.data_converter import generate_asp_dataset
from scripts.data_converter_2 import generate_asp_dataset_2
from scripts.data_converter_3 import generate_asp_dataset_3
from scripts.data_converter_4 import generate_asp_dataset_4
from scripts.data_converter_maximize_data import generate_asp_dataset_max

# Define paths
root = get_root_path()
data_root = os.path.join(root, "Finetuning", "data", "Benchmark Data")
output_dir_oud = os.path.join(root, "Finetuning", "data", "oud")
output_dir_1 = os.path.join(root, "Finetuning", "data", "een_op_een")
output_dir_zonder_syntax_fouten = os.path.join(root, "Finetuning", "data", "best_zonder_syntax")
output_dir_niet_maximaal = os.path.join(root, "Finetuning", "data", "niet_maximaal")
output_dir = os.path.join(root, "Finetuning", "data")

print(f"Input: {to_relative(data_root)}")
print(f"Output old: {to_relative(output_dir_oud)}")
print(f"Output 1: {to_relative(output_dir_1)}")
print(f"Output beste zonder syntax dingen: {to_relative(output_dir_zonder_syntax_fouten)}")
print(f"Output niet maximaal: {to_relative(output_dir_niet_maximaal)}")
print(f"Output: {to_relative(output_dir)}")

Input: Finetuning\data\Benchmark Data
Output old: Finetuning\data\oud
Output 1: Finetuning\data\een_op_een
Output beste zonder syntax dingen: Finetuning\data\best_zonder_syntax
Output niet maximaal: Finetuning\data\niet_maximaal
Output: Finetuning\data


In [2]:
# Generate the data one-shot like from NL -> ASP and IR -> ASP
print(f"Processing data from: ./{to_relative(data_root)}")
train_count, valid_count = generate_asp_dataset(data_root, output_dir_oud)

print(f"Success!")
print(f"Train examples: {train_count}")
print(f"Validation examples: {valid_count}")
print(f"Files saved in: ./{to_relative(output_dir_oud)}")

Processing data from: ./Finetuning\data\Benchmark Data
Success!
Train examples: 108
Validation examples: 27
Files saved in: ./Finetuning\data\oud


In [3]:
# This now generates multi-turn conversations based on comments in the ASP files
train_count, valid_count = generate_asp_dataset_2(data_root, output_dir_1)

print(f"\nSuccess!")
print(f"Training Examples: {train_count}")
print(f"Validation Examples: {valid_count}")
print(f"Files saved in: ./{to_relative(output_dir_1)}")

Traversing c:\Users\arjan\SchASPLM-Shamantics\Finetuning\data\Benchmark Data...
Found 108 problem instances.

Success!
Training Examples: 86
Validation Examples: 22
Files saved in: ./Finetuning\data\een_op_een


In [4]:
# This now generates multi-turn conversations based on comments in the ASP files
# This also generates variants of every example to prevent overfitting
train_count, valid_count = generate_asp_dataset_3(data_root, output_dir_zonder_syntax_fouten, split_ratio=0.8)

print(f"\nSuccess!")
print(f"Training Examples: {train_count}")
print(f"Validation Examples: {valid_count}")
print(f"Files saved in: ./{to_relative(output_dir_zonder_syntax_fouten)}")

Traversing c:\Users\arjan\SchASPLM-Shamantics\Finetuning\data\Benchmark Data...
Found 108 unique problem instances.
Splitting: 86 problems for Training, 22 for Validation.

Success!
Training Examples: 430
Validation Examples: 110
Files saved in: ./Finetuning\data\best_zonder_syntax


In [5]:
# This now generates multi-turn conversations based on comments in the ASP files
# This also generates variants of every example to prevent overfitting
train_count, valid_count = generate_asp_dataset_4(data_root, output_dir_niet_maximaal, split_ratio=0.8)

print(f"\nSuccess!")
print(f"Training Examples: {train_count}")
print(f"Validation Examples: {valid_count}")
print(f"Files saved in: ./{to_relative(output_dir_niet_maximaal)}")

Traversing c:\Users\arjan\SchASPLM-Shamantics\Finetuning\data\Benchmark Data...
Found 108 unique problem instances.
Splitting: 86 Train, 22 Valid.

Success!
Training Examples: 516
Validation Examples: 132
Files saved in: ./Finetuning\data\niet_maximaal


In [6]:
# This now generates multi-turn conversations based on comments in the ASP files
# This also generates variants of every example to prevent overfitting
train_count, valid_count = generate_asp_dataset_max(data_root, output_dir, split_ratio=0.8)

print(f"\nSuccess!")
print(f"Training Examples: {train_count}")
print(f"Validation Examples: {valid_count}")
print(f"Files saved in: ./{to_relative(output_dir)}")

Traversing c:\Users\arjan\SchASPLM-Shamantics\Finetuning\data\Benchmark Data...
Found 27 unique logical problems (spanning 108 folders).
Splitting: 21 Problem Groups for Training, 6 for Validation.

Success!
Training Examples: 1197
Validation Examples: 288
Files saved in: ./Finetuning\data


In [7]:
train_file = os.path.join(output_dir, "train.jsonl")

data_path = os.path.join("..", "Finetuning", "data", "train.jsonl")

# 1. Load all lines into memory (efficient enough for small-medium datasets)
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 2. Pick a random line
if lines:
    random_line = random.choice(lines)
    sample = json.loads(random_line)

    print(f"--- RANDOM SAMPLE (Total Examples: {len(lines)}) ---")
    
    # 3. Iterate through messages and print nicely
    for msg in sample['messages']:
        role = msg['role'].upper()
        content = msg['content']
        
        print(f"\n[{role}]:")
        print("-" * 20)
        print(content)
        print("=" * 60)
else:
    print("Dataset file is empty.")

--- RANDOM SAMPLE (Total Examples: 1197) ---

[SYSTEM]:
--------------------
You are an expert Clingo code debugger. Fix the syntax errors in the following ASP rules.

[USER]:
--------------------
The following code has an error:
:- schedule(1, R, T), R != 1

Clingo Error:
error: syntax error, unexpected <EOF>, expecting .

Fix the code.

[ASSISTANT]:
--------------------
:- schedule(1, R, T), R != 1.
